In [12]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math
from qutip import *
import random
from math import pi
# Qiskit modules
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, IBMQ
from qiskit import execute
import qiskit.tools.visualization
from qiskit.tools.visualization import circuit_drawer, plot_histogram
from qiskit.quantum_info import state_fidelity
from multiprocess import Process, Manager,Lock


In [13]:
cycles=35

c=0.0299792 #speed of light in units cm/ps

g1=300*c*2*pi  # fluctuation strength, in units rad.THz, of the first fluctuator-pair 

energy  = [2466.33, 2416.88, 2327.67, 2280.44]
init=False
coupling_values=[g1] 
precision=len(coupling_values) # number of fluctuator-pairs
dec_times=[0.008, 0.008]  
P = 50 # number of processes
N = 5 # number of different algorithm runs in each process
PN=P*N  # number of total runs of the algorithm
shots=5000

In [14]:

# list of coupling strengths, add more, to have more fluctuators acting identically in each molecule. 
# For instance, one element, denotes two identical fluctuators acting each, in one of the molecules (fluctuator-pair). 
# Index "i" corresponds to the fluctuator waiting time of the index "i+1" in "dec_times".
def state_initialization_random(cir,sys,anc,init):
    cir.x(anc[1])
    if (init):
        cir.x(sys[1])
        cir.x(sys[0])

def graphic(PN,iterations,dec_times,P,N,E1,E2,E3,E4,init,coupling_values,precision,shots):
    #  Required variable delcarations
    x=[]
    y1=[]
    y2=[]
    y3=[]
    y4=[]
    p1=0
    p2=0
    p3=0
    p4=0
    for cycles in range(0,iterations):
        print(cycles)
        if __name__ == '__main__':  # for each simulation for a time t, create P processes
            manager=Manager()
            my_l = Lock()
            processes=[]
            V = manager.dict()
            V['00']=0
            V['01']=0
            V['10']=0
            V['11']=0
            for i in range(0,P):
                p = Process(target = execute_N,args=(my_l,V,N,energy[0],energy[1],energy[2],energy[3],dec_times,cycles,init,coupling_values,precision,shots))
                processes.append(p)
            for u in processes:
                u.start()
            for u in processes:
                u.join()
            # Get the population terms (probability), divide by the number of the total algorithm runs
            p1+=V['00']/PN
            p2+=V['01']/PN
            p3+=V['10']/PN
            p4+=V['11']/PN
        x.append(dec_times[0]*cycles)
        y1.append(p1)
        y2.append(p2)
        y3.append(p3)
        y4.append(p4)
        p1=0
        p2=0
        p3=0
        p4=0

    
    la = plt.scatter(x,y1,c='b',marker="^",label='Simulation - P(0)')
    lb = plt.scatter(x,y2,c='r',label='Simulation - P(1)')
    lc = plt.scatter(x,y3,c='g',marker="s", label='Simulation - P(2)')
    ld = plt.scatter(x,y4,c="black",marker="X",label='Simulation - P(3)')
    lx = plt.xlabel('Time (ps)')
    ly = plt.ylabel('Probability')
    xx = plt.xlim(0,cycles*dec_times[0])
    ll = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

In [15]:
def rzx(cir, angx, angz, cqub, qub):
    cir.crz(2*angz, cqub, qub)
    cir.crx(angx, cqub, qub)

def ctrzx(cir, angx, angz, cqub, qub):
    cir.crx(-angx, cqub, qub)
    cir.crz(-2*angz, cqub, qub)
    
def phase(cir, cont, anc, ang):
    cir.ccx(cont[0], cont[1], anc[0])
    cir.crz(2*ang, anc[0], anc[1])
    cir.ccx(cont[0], cont[1], anc[0])
    
def ctphase(cir, cont, anc, ang):
    cir.ccx(cont[0], cont[1], anc[0])
    cir.crz(-2*ang, anc[0], anc[1])
    cir.ccx(cont[0], cont[1], anc[0])  

In [16]:
# execute N different runs of the quantum algorithm in each process:
def execute_N(my_l,V,N,E1,E2,E3,E4,dec_times,cycles,init,coupling_values,precision,shotss):
    backend_sim = Aer.get_backend('qasm_simulator')
    coupling_values_=[]
    dec_times_=[]
    # Concurrency control with a lock "my_l": create new variables in each process to read the input parameters
    my_l.acquire()
    shots_=shotss
    E1_=E1
    E2_=E2
    E3_=E3
    E4_=E4
    init_=init
    precision_=precision
    cycles_=cycles
    N_=N
    dec_times_.append(dec_times[0])
    for i in range(precision):
        coupling_values_.append(coupling_values[i])
        dec_times_.append(dec_times[i+1])
    my_l.release()
    
    # For each run, create a new quantum circuit and new random values
    for i in range(N_):
        exc = QuantumRegister(2)
        out = QuantumRegister(2)
        cr = ClassicalRegister(2)
        nat = QuantumCircuit(exc,out,cr)
        
        random_values = random_generator(cycles_,precision_)
        two_qubit_nat_random(nat,exc,out,cr,E1_,E2_,E3_,E4_,dec_times_,cycles_,init_,random_values,coupling_values_,precision_)
        job_sim = execute(nat, backend_sim, shots=shots_)
        result_sim = job_sim.result()
        result_counts = result_sim.get_counts(nat)
        
        # Concurrency control : Add the probability of measuring each state "k" in each run to a global dictionary
        # (which can be utilized by every process). In function "graphic", will be divided by "PN" 
        # (the number of all runs of the simulation).
        my_l.acquire()
        for k in result_counts:
            V[k]+=result_counts[k]/shots_
        my_l.release()
        
        
        

In [17]:
def ctubtCir(cir, sys, anc):
    cir.x(sys[1])
    ctrzx(cir, -1.2808427, -1.5707964, sys[1], sys[0])
    cir.x(sys[1])
    cir.barrier()
    
    cir.x(sys[0])
    ctrzx(cir, -0.2392549, -0.7853982, sys[0], sys[1])
    cir.x(sys[0])
    cir.barrier()
    
    cir.x(sys[1])
    cir.cx(sys[1], sys[0])
    cir.x(sys[1])
    ctrzx(cir, -0.63581735, -0.39269906, sys[0], sys[1])
    cir.x(sys[1])
    cir.cx(sys[1], sys[0])
    cir.x(sys[1])
    cir.barrier()

    cir.cx(sys[0], sys[1])
    ctrzx(cir, -0.37570226, -1.9634955, sys[1], sys[0])
    cir.cx(sys[0], sys[1])
    cir.barrier()
    
    ctrzx(cir, -0.1030699, -1.1780972, sys[0], sys[1])
    cir.barrier()
    
    ctrzx(cir, -1.2680272, -2.159845, sys[1], sys[0])
    cir.barrier()
    
    ctphase(cir, sys, anc, 1.8653208)
    cir.barrier()
    
    cir.x(sys[0])
    ctphase(cir, sys, anc, -2.8470685)
    cir.x(sys[0])
    cir.barrier()
    
    cir.x(sys[1])
    ctphase(cir, sys, anc, 2.3561945)
    cir.x(sys[1])
    cir.barrier()
    
    cir.x(sys[0])
    cir.x(sys[1])
    ctphase(cir, sys, anc, 1.767146)
    cir.x(sys[1])
    cir.x(sys[0])
    cir.barrier()
    cir.barrier()

In [18]:
def evolution(cir, sys, anc, t, E1,E2,E3,E4):
    cir.x(sys[0:2])
    cir.ccx(sys[0], sys[1], anc[0])
    cir.crz(-E4*t, anc[0],anc[1])
    cir.ccx(sys[0], sys[1], anc[0])
    cir.x(sys[0:2])
    cir.barrier()

    cir.x(sys[1])
    cir.ccx(sys[0], sys[1], anc[0])
    cir.crz(-E2*t, anc[0], anc[1])
    cir.ccx(sys[0], sys[1], anc[0])
    cir.x(sys[1])
    cir.barrier()

    cir.x(sys[0])
    cir.ccx(sys[0], sys[1], anc[0])
    cir.crz(-E3*t, anc[0],anc[1])
    cir.ccx(sys[0], sys[1], anc[0])
    cir.x(sys[0])
    cir.barrier()


    cir.ccx(sys[0], sys[1], anc[0])
    cir.crz(-E4*t, anc[0],anc[1])
    cir.ccx(sys[0], sys[1], anc[0])
    cir.barrier()    
    cir.barrier()

In [19]:
def ubtCir(cir, sys, anc):
    cir.x(sys[1])
    cir.x(sys[0])
    phase(cir, sys, anc, 1.767146)
    cir.x(sys[0])
    cir.x(sys[1])
    cir.barrier()
    
    cir.x(sys[1])
    phase(cir, sys, anc, 2.3561945)
    cir.x(sys[1])
    cir.barrier()
    
    cir.x(sys[0])
    phase(cir, sys, anc, -2.8470685)
    cir.x(sys[0])
    cir.barrier()
    
    phase(cir, sys, anc, 1.8653208)
    cir.barrier()
    
    rzx(cir, -1.2680272, -2.159845, sys[1], sys[0])
    cir.barrier()
    
    rzx(cir, -0.1030699, -1.1780972, sys[0], sys[1])
    cir.barrier()
    
    cir.cx(sys[0], sys[1])
    rzx(cir, -0.37570226, -1.9634955, sys[1], sys[0])
    cir.cx(sys[0], sys[1])
    cir.barrier()
    
    cir.x(sys[1])
    cir.cx(sys[1], sys[0])
    cir.x(sys[1])
    rzx(cir, -.063581735, -0.39269906, sys[0], sys[1])
    cir.x(sys[1])
    cir.cx(sys[1], sys[0])
    cir.x(sys[1])
    cir.barrier()
    
    cir.x(sys[0])
    rzx(cir, -0.2392549, -0.7853982, sys[0], sys[1])
    cir.x(sys[0])
    cir.barrier()
    
    cir.x(sys[1])
    rzx(cir, -1.2808427, -1.5707964, sys[1], sys[0])
    cir.x(sys[1])
    cir.barrier()
    cir.barrier()

In [20]:
def two_qubit_nat_random(nat,exc,out,cr,E1,E2,E3,E4,dec_times,cycles,init,random_values,coupling_values,precision):
    index_random = []  
    # the index of this list, denotes a fluctuator-pair. The value correspondent to an index "i" denotes 
    # the index of the random number in the list of lists of random numbers "randl", i.e. randl[i][index_random[i]]
    # points to a new random number correspondent to the fluctuator-pair "i".
    
    for i in range(len(coupling_values)): # all fluctuator-pairs start at the index 0.
        index_random.append(0)
        
    state_initialization_random(nat,exc,out,init)
    nat.barrier()
    for i in range(0, cycles): # iteration procedure (required for Trotter product formula)
        ubtCir(nat,exc,out)
        evolution(nat,exc,out,dec_times[0],E1,E2,E3,E4)
        nat.barrier()
        ctubtCir(nat,exc,out)
        nat.barrier()
        index_random = system_bath_evolution_random(nat,exc,out,i,dec_times,random_values,index_random,coupling_values,precision)
        # index random as input and output at every iteration to keep its current value.
        nat.barrier()
    nat.measure(exc,cr)

In [21]:
def random_generator(cycles,precision):
    n_values = cycles*4 # number of required random numbers a fluctuator-pair
    values = ''     # list of random numbers for each fluctuator-pair
    randl=[]                # list containing all the fluctuator-pair random number lists 
                            # (because there can be more than one fluctuator in each set)
    # create the random numbers and add them to the respective lists
    for j in range(precision): 
        for i in range(n_values):
            values += str(random.randint(0,1))
        randl.append(values)
        values=''
    return randl

def system_bath_evolution_random(nat,exc,out,i,dec_times,random_values,index_random,coupling_values,precision):
    # 00
    nat.x(exc[0])
    nat.x(exc[1])
    nat.ccx(exc[0], exc[1], out[0])
    index_random = apply_single_h_random(nat,exc,out,random_values,index_random,i,dec_times,coupling_values,precision,1)
    nat.ccx(exc[0], exc[1], out[0])
    nat.x(exc[1])
    nat.x(exc[0])
    #01
    nat.x(exc[1])
    nat.ccx(exc[0], exc[1], out[0])
    index_random = apply_single_h_random(nat,exc,out,random_values,index_random,i,dec_times,coupling_values,precision,2)
    nat.ccx(exc[0], exc[1], out[0])
    nat.x(exc[1])
    #10
    nat.x(exc[0])
    nat.ccx(exc[0], exc[1], out[0])
    index_random = apply_single_h_random(nat,exc,out,random_values,index_random,i,dec_times,coupling_values,precision,3)
    nat.ccx(exc[0], exc[1], out[0])
    nat.x(exc[0])
    #11
    nat.ccx(exc[0], exc[1], out[0])
    index_random = apply_single_h_random(nat,exc,out,random_values,index_random,i,dec_times,coupling_values,precision,4)
    nat.ccx(exc[0], exc[1], out[0])

    return(index_random)

def apply_single_h_random(nat,exc,out,random_values,index_random,j,dec_times,coupling_values,precision,n_mol):
    # "j" denotes the current cycle, "dec_times[0]" the iteration time-step.
    for i in range(precision): # "i" denotes a fluctuator-pair
        if((j*dec_times[0])%dec_times[i+1]==0.0): # if it is time to apply a new random value (switching act)
            
            if(random_values[i][index_random[i]]=='0'):          # if '0', then apply sign -
                nat.crz(-coupling_values[i]*dec_times[0],out[0],out[1])
            elif(random_values[i][index_random[i]]=='1'):        # if '1', then apply sign +
                nat.crz(coupling_values[i]*dec_times[0],out[0],out[1])
            index_random[i] = index_random[i]+1         # increment the index which points to a new random value
            
        else:                               # if it is not time to apply a switching act 
                                            # (do not increment the index which points to a new random value)
            
            if(n_mol==1):  # i.e. donor molecule, so get the random value used in its last interaction with the 
                           # respective fluctuator in the fluctuator-pair "i"
                if(random_values[i][index_random[i]-4]=='0'):   
                        nat.crz(-coupling_values[i]*dec_times[0],out[0],out[1])
                elif(random_values[i][index_random[i]-4]=='1'):
                        nat.crz(coupling_values[i]*dec_times[0],out[0],out[1])
                        
            elif(n_mol==2):          # if acceptor molecule, get the random value used in its last interaction with the 
                           # respective fluctuator in the fluctuator-pair "i"
                if(random_values[i][index_random[i]-3]=='0'):
                        nat.crz(-coupling_values[i]*dec_times[0],out[0],out[1])
                elif(random_values[i][index_random[i]-3]=='1'):
                        nat.crz(coupling_values[i]*dec_times[0],out[0],out[1])
            
            elif(n_mol==3):          # if acceptor molecule, get the random value used in its last interaction with the 
                           # respective fluctuator in the fluctuator-pair "i"
                if(random_values[i][index_random[i]-2]=='0'):
                        nat.crz(-coupling_values[i]*dec_times[0],out[0],out[1])
                elif(random_values[i][index_random[i]-2]=='1'):
                        nat.crz(coupling_values[i]*dec_times[0],out[0],out[1])
                        
            elif(n_mol==4):          # if acceptor molecule, get the random value used in its last interaction with the 
                           # respective fluctuator in the fluctuator-pair "i"
                if(random_values[i][index_random[i]-1]=='0'):
                        nat.crz(-coupling_values[i]*dec_times[0],out[0],out[1])
                elif(random_values[i][index_random[i]-1]=='1'):
                        nat.crz(coupling_values[i]*dec_times[0],out[0],out[1])
                        
    return(index_random) # return the index of the new random number


In [ ]:
graphic(PN,cycles,dec_times,P,N,energy[0],energy[1],energy[2],energy[3],init,coupling_values,precision,shots)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [ ]:
s